# Analyzing Rubik's Cube with GAP

This is a GAP.jl adaptation of the [Rubik's Cube example](https://www.gap-system.org/Doc/Examples/rubik.html) originally written by Martin Schönert.

We begin by defining a few convenient shorthands to access GAP.jl functionality. While it is generally sufficient and convenient to use the `@gap` macro to enter permutations, e.g. `@gap "(1, 3)"`, we add functions here that allow us to do that in a typesafe fashion.

In [1]:
using GAP

G = GAP.Globals
to_gap = GAP.julia_to_gap
from_gap = GAP.gap_to_julia
gap_eval = GAP.EvalString

perm(cycle::Int...) = gap_eval(string("(", join(cycle, ","), ")"))
perm(cycles::Tuple{Vararg{Int}}...) =
  gap_eval(join([ string("(", join(c, ","), ")") for c in cycles ]))
big_int(x) = BigInt(from_gap(x));

In this example, we consider the group of transformations of Rubik's magic cube. If we number the faces of this cube as follows

                     +--------------+
                     |              |
                     |  1    2    3 |
                     |              |
                     |  4  top    5 |
                     |              |
                     |  6    7    8 |
                     |              |
      +--------------+--------------+--------------+--------------+
      |              |              |              |              |
      |  9   10   11 | 17   18   19 | 25   26   27 | 33   34   35 |
      |              |              |              |              |
      | 12  left  13 | 20 front  21 | 28 right  29 | 36  rear  37 |
      |              |              |              |              |
      | 14   15   16 | 22   23   24 | 30   31   32 | 38   39   40 |
      |              |              |              |              |
      +--------------+--------------+--------------+--------------+
                     |              |
                     | 41   42   43 |
                     |              |
                     | 44 bottom 45 |
                     |              |
                     | 46   47   48 |
                     |              |
                     +--------------+

then the group is generated by the following generators, corresponding to the six faces of the cube.



In [2]:
cube = @gap "Group(
       ( 1, 3, 8, 6)( 2, 5, 7, 4)( 9,33,25,17)(10,34,26,18)(11,35,27,19),
       ( 9,11,16,14)(10,13,15,12)( 1,17,41,40)( 4,20,44,37)( 6,22,46,35),
       (17,19,24,22)(18,21,23,20)( 6,25,43,16)( 7,28,42,13)( 8,30,41,11),
       (25,27,32,30)(26,29,31,28)( 3,38,43,19)( 5,36,45,21)( 8,33,48,24),
       (33,35,40,38)(34,37,39,36)( 3, 9,46,32)( 2,12,47,29)( 1,14,48,27),
       (41,43,48,46)(42,45,47,44)(14,22,30,38)(15,23,31,39)(16,24,32,40) )"

GAP: <permutation group with 6 generators>

First we want to know the size of this group.

In [3]:
G.Size(cube)

GAP: 43252003274489856000

Since this is a little bit unhandy, let us factorize this number.

In [4]:
G.Collected(G.Factors(G.Size(cube)))

GAP: [ [ 2, 27 ], [ 3, 14 ], [ 5, 3 ], [ 7, 2 ], [ 11, 1 ] ]

(The result tells us that the size is 2<sup>27</sup>3<sup>14</sup>5<sup>3</sup>7<sup>2</sup>11.)

Next let us investigate the operation of the group on the 48 points (we reduce the line length to get a more appropriate output format).

In [5]:
orbits = G.Orbits(cube, to_gap(1:48))

GAP: [ [ 1, 3, 17, 14, 8, 38, 9, 41, 19, 48, 22, 6, 30, 33, 43, 11, 46, 40, 24, 
      27, 25, 35, 16, 32 ], 
  [ 2, 5, 12, 7, 36, 10, 47, 4, 28, 45, 34, 13, 29, 44, 20, 42, 26, 21, 37, 
      15, 31, 18, 23, 39 ] ]

The first orbit contains the points at the corners, the second those at the edges; clearly the group cannot move a point at a corner onto a point at an edge.

So to investigate the cube group we first investigate the operation on the corner points. Note that the constructed group that describes this operation will operate on the set `[1..24]`, not on the original set `[1,3,17,14,8,38,9,41,19,48,22,6,30,33,43,11,46,40,24,27,25,35,16,32]`.

In [6]:
cube1 = G.Action(cube, orbits[1])

GAP: <permutation group with 6 generators>

In [7]:
G.NrMovedPoints(cube1)

24

In [8]:
G.Size(cube1)

88179840

Now this group obviously operates transitively, but let us test whether it is also primitive.

In [9]:
corners = G.Blocks(cube1, G.MovedPoints(cube1))

GAP: [ [ 1, 7, 22 ], [ 2, 14, 20 ], [ 3, 12, 16 ], [ 4, 17, 18 ], [ 5, 9, 21 ], 
  [ 6, 10, 24 ], [ 8, 11, 23 ], [ 13, 15, 19 ] ]

Those eight blocks correspond to the eight corners of the cube; on the one hand the group permutes those and on the other hand it permutes the three points at each corner cyclically.

So the obvious thing to do is to investigate the operation of the group on the eight corners. The action gives a homomorphism to a permutation group on the corners:

In [10]:
blockhom1 = G.ActionHomomorphism( cube1, corners, G.OnSets )

GAP: <action homomorphism>

In [11]:
cube1b = G.Image(blockhom1)

GAP: Group([ (1,2,4,3), (1,3,6,5), (1,5,8,2), (3,4,7,6), (5,6,7,8), (2,8,7,4) ])

In [12]:
G.Size(cube1b)

40320

Now a permutation group of degree 8 that has order 40320 must be the full symmetric group S(8) on eight points.

The next thing then is to investigate the kernel of this operation on blocks, i.e., the subgroup of cube1 of those elements that fix the blocks setwise.

In [13]:
G.Factors(G.Size(G.Kernel(blockhom1)))

GAP: [ 3, 3, 3, 3, 3, 3, 3 ]

In [14]:
G.IsElementaryAbelian(G.Kernel(blockhom1))

true

We can show that the product of this elementary abelian group 3<sup>7</sup> with the S(8) is semidirect by finding a complement, i.e., a subgroup that has trivial intersection with the kernel and that generates cube1 together with the kernel.

In [15]:
cmpl1 = G.Complementclasses(cube1, G.Kernel(blockhom1))

GAP: [ <permutation group with 7 generators> ]

In [16]:
cmpl1 = cmpl1[1]
G.Size(cmpl1)

40320

We verify the complement properties:

In [17]:
G.Size(G.Intersection(cmpl1, G.Kernel(blockhom1)))

1

In [18]:
G.ClosureGroup(cmpl1, G.Kernel(blockhom1)) == cube1

true

There is even a more elegant way to show that `cmpl1` is a complement.

In [19]:
G.IsBijective(G.RestrictedMapping(blockhom1, cmpl1))

true

Of course, theoretically it is clear that cmpl1 must indeed be a complement.

In fact we know that cube1 is a subgroup of index 3 in the wreath product of a cyclic 3 with S(8). This missing index 3 tells us that we do not have total freedom in turning the corners. The following tests show that whenever we turn one corner clockwise we must turn another corner counterclockwise.

In [20]:
G.in(perm(1, 7, 22), cube1)

false

In [21]:
G.in(perm((1, 7, 22),(2, 20, 14)), cube1)

true

More or less the same things happen when we consider the operation of the cube group on the edges.

In [22]:
cube2 = G.Action(cube, orbits[2]);
G.Size(cube2)

980995276800

In [23]:
edges = G.Blocks(cube2, G.MovedPoints(cube2))

GAP: [ [ 1, 11 ], [ 2, 17 ], [ 3, 19 ], [ 4, 22 ], [ 5, 13 ], [ 6, 8 ], [ 7, 24 ], 
  [ 9, 18 ], [ 10, 21 ], [ 12, 15 ], [ 14, 20 ], [ 16, 23 ] ]

In [24]:
blockhom2 = G.ActionHomomorphism(cube2, edges, G.OnSets)
cube2b = G.Image(blockhom2)
G.Size(cube2b)

479001600

In [25]:
G.Factors(G.Size(G.Kernel(blockhom2)))

GAP: [ 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2 ]

In [26]:
G.IsElementaryAbelian(G.Kernel(blockhom2))

true

In [27]:
cmpl2 = G.Complementclasses(cube2, G.Kernel(blockhom2));
length(cmpl2)

4

So there are even 4 classes of complements here. This time we get a semidirect product of a 2^11 with an S(12), namely a subgroup of index 2 of the wreath product of a cyclic 2 with S(12). Here the missing index 2 tells us again that we do not have total freedom in turning the edges. The following tests show that whenever we flip one edge we must also flip another edge.

In [28]:
G.in(perm(1, 11), cube2)

false

In [29]:
G.in(perm((1, 11),(2, 17)), cube2)

true

Since `cube1` and `cube2` are the groups describing the actions on the two orbits of cube, it is clear that cube is a subdirect product of those groups, i.e., a subgroup of the direct product. Comparing the sizes of `cube1`, `cube2`, and `cube` we see that `cube` must be a subgroup of index 2 in the direct product of those two groups.

Note that in Julia, we must use big integer arithmetic, as the values exceed 64 bits.

In [30]:
big_int(G.Size(cube1)) * big_int(G.Size(cube2))

86504006548979712000

In [31]:
big_int(G.Size(cube))

43252003274489856000

This final missing index 2 tells us that we cannot operate on corners and edges totally independently. The following tests show that whenever we exchange a pair of corners we must also exchange a pair of edges (and vice versa).

In [32]:
G.in(perm((17, 19), (11, 8), (6, 25)), cube)

false

In [33]:
G.in(perm((7, 28), (18, 21)), cube)

false

In [34]:
G.in(perm((17,19), (11,8), (6,25), (7,28), (18,21)), cube)

true

As a last part of the structure analysis of the cube group let us compute the centre of the cube group, i.e., the subgroup of those operations that can be performed either before or after any other operation with the same result.

In [35]:
z = G.Centre(cube)

GAP: Group([ (2,34)(4,10)(5,26)(7,18)(12,37)(13,20)(15,44)(21,28)(23,42)(29,36)
  (31,45)(39,47) ])

We see that the centre contains one nontrivial element, namely the operation that flips all 12 edges simultaneously.

Finally we turn to the original idea connected with the cube, namely to find a sequence of turns of the faces that will transform the cube back into its original state. This amounts to a decomposition of a given element of the cube group into a product of the generators. For this purpose we introduce a free group and a homomorphism of it onto the cube group.

We use the `g"string"` notation to enter GAP strings in Julia.

In [36]:
f = G.FreeGroup(g"t", g"l", g"f", g"r", g"e", g"b")

GAP: <free group on the generators [ t, l, f, r, e, b ]>

In [37]:
hom = G.GroupHomomorphismByImages(f, cube, G.GeneratorsOfGroup(f), G.GeneratorsOfGroup(cube))

GAP: [ t, l, f, r, e, b ] -> 
[ (1,3,8,6)(2,5,7,4)(9,33,25,17)(10,34,26,18)(11,35,27,19), 
  (1,17,41,40)(4,20,44,37)(6,22,46,35)(9,11,16,14)(10,13,15,12), 
  (6,25,43,16)(7,28,42,13)(8,30,41,11)(17,19,24,22)(18,21,23,20), 
  (3,38,43,19)(5,36,45,21)(8,33,48,24)(25,27,32,30)(26,29,31,28), 
  (1,14,48,27)(2,12,47,29)(3,9,46,32)(33,35,40,38)(34,37,39,36), 
  (14,22,30,38)(15,23,31,39)(16,24,32,40)(41,43,48,46)(42,45,47,44) ]

Using this homomorphism, we can now decompose elements into generators. The method used utilizes a stabilizer chain and does not enumerate all group elements, therefore the words obtained are not the shortest possible, though they are short enough for hand solutions.

First we decompose the centre element:

In [38]:
pre1 = G.PreImagesRepresentative(hom, G.GeneratorsOfGroup(z)[1])

GAP: l*f*t*f^-1*t^-1*l^-1*f*t*f^2*l*f*l^-2*t*l*f^-1*t^-1*l^-1*t*l*f^2*t*l*t*l^-1*f^\
-1*l*t^-1*l^-1*f*r*t^-1*r^-1*f^-1*l*t*f*t^-1*f^-1*l^-1*t*f^-1*l^-1*t^-1*l*t*f*\
t^-2*f*t*f^-1*t^-1*l^-1*t^-1*l*t^-1*l^-1*t*e^-1*t*e*(l*t)^2*e*l^-1*e^-1*t^-1*l\
^-3*b*f*b^-1*l^-1*f^-1*t*l^-1*f*t*f*l^-1*t^-1*b*r^-1*b^-1*t^-2*e^-1*r*e*r*f^-1\
*e*t^-1*e^-1*r^-2*t^-2*l^-1*b^-1*r^-1*e^-1

In [39]:
length(pre1)

108

Note that we can use Julia's `length` function to directly operate on `pre1`.

There are also alternate ways to access the only generator of `z` using GAP's `z.1` notation, as done in the original GAP code. We can either construct a GAP function that can be called in Julia using `@gap` or we can use `z.:1`.

In [40]:
pre1 == G.PreImagesRepresentative(hom, @gap("x -> x.1")(z))

true

In [41]:
pre1 == G.PreImagesRepresentative(hom, z.:1)

true

Next we decompose some element arbitrarily chosen by us:

In [42]:
pre2 = G.PreImagesRepresentative(hom, perm((17, 19), (11, 8), (6, 25), (7, 28), (18, 21)))

GAP: l^-1*t^-1*l*f*r*t*r^-1*f^-1*l*t*f*t^-1*f^-1*l^-1*t^2*f*t*l*t*l^-1*f^-1*l*t^-1*\
l^-1*f*t^-1*f^-1*l*t*l^-1*t*l*t^-2*l^-1*f*(t*r*t^-1*r^-1)^2*f^-1*t*l*f^-1*l^-1\
*f*l^-1*t^-1*l*t^-2*f*t*(f^-1*l^-1)^2*l^-1*f*l*e^-1*t*e*l*t^-1*e^-1*t^-1*e*l*b\
*f^-1*b^-1

In [43]:
length(pre2)

77

Last we let GAP choose a random element ...

In [44]:
r = G.Random(cube)

GAP: (1,9,35)(2,26,42,28,47,44,13,12,31,34,5,23,21,39,15,20,37,45)(3,19,46,11,41,
33,8,40,6,16,27,25,14,17,22)(4,18,36,10,7,29)(32,38,48)

In [45]:
pre3 = G.PreImagesRepresentative(hom, r)

GAP: f*b^2*f^-1*r^-1*l^-1*e*b^-2*e^-1*t^2*b^-1*e^-1*b*f*t^2*f^-1*l*t^-1*f*l^-1*b*f^\
-2*b^-1*l^-1*e^-1*t*e*l^-1*f^-1*l*f*t^-1*(t^-1*l^-1)^2*f^-1*l^-1*f*l*f*t*f^-1*\
t^-1*l*f^-1*l^-1*f*l^-1*(t^-1*l)^2*f*t*f^-1*t^-1*l^-1*t*l*t^2*l^-1*t^-1*l*t^-1\
*l^-1*t*f*r*t*r^-1*t^-1*f^-1*l^-1*t^-1*(l*t)^2*f^-1*l^-1*e^-1*f*t^-1*e*l*t^-1*\
l^-1*t

In [46]:
length(pre3)

94

... and we verify that the decomposition is correct:

In [47]:
img3 = G.Image(hom, pre3)

GAP: (1,9,35)(2,26,42,28,47,44,13,12,31,34,5,23,21,39,15,20,37,45)(3,19,46,11,41,
33,8,40,6,16,27,25,14,17,22)(4,18,36,10,7,29)(32,38,48)

In [48]:
img3 == r

true